In [1]:
# want to extract 
# 1) when an article is published at 
# 2) funding round, 
# 3) money raied

In [2]:
import pandas as pd
import re
data = pd.read_csv("article.csv", encoding='iso-8859-1')

In [3]:
data.head(10)

,title,link,excerpt
0,Indian online lending platform Capital Float r...,https://techcrunch.com/2017/08/21/indian-onlin...,"Capital Float, which claims to be the largest ..."
1,"Carwow, a UK startup that helps you buy a new ...",https://techcrunch.com/2017/07/30/carwow-serie...,"Carwow, a platform that helps you buy a new ca..."
2,Stash raises $40 million Series C to make inve...,https://techcrunch.com/2017/07/13/stash-raises...,Micro-investing app Stash has raised an additi...
3,"Blowing up the re-location industry, Move Guid...",https://techcrunch.com/2017/07/11/blowing-up-t...,Whatever any country leader says about banning...
4,Cohesity raises $90M+ Series C round for its h...,https://techcrunch.com/2017/04/04/cohesity-rai...,"Cohesity, a company that helps enterprises to ..."
5,Digg raises Series C led by USA Today owner Ga...,https://techcrunch.com/2016/09/13/digg-raises-...,As publishers of dead-tree newspapers continue...
6,"After bump in the road, Movinga raises 17M Se...",https://techcrunch.com/2016/12/08/on-the-movin...,Berlin's startup scene is probably the most go...
7,Leanplum raises $29M Series C round for its mo...,https://techcrunch.com/2016/10/19/leanplum-rai...,Leanplum started out as a mobile A/B testing p...
8,"Forter raises $32M Series C, continues battle ...",https://techcrunch.com/2016/04/21/forter-serie...,Fraud prevention company Forter today announce...
9,MemSQL raises $36M Series C round for its in-m...,https://techcrunch.com/2016/04/21/memsql-raise...,In-memory database platform MemSQL today annou...


In [4]:
# drop separator rows
data = data[data.title != "title"]

In [5]:
data.shape

(1066, 3)

In [6]:
# get published_at info
def get_dates(x):
    try:
        return re.search(r'^https:\/\/techcrunch\.com\/(\d{4}\/\d{2}\/\d{2})', x).group(1)
    except:
        return 0
    
# new column "publiched_at"
data["published_at"] = data.link.apply(get_dates)

In [7]:
# remove article without published ate
data = data[data.published_at != 0]

In [8]:
# get funding_round info
def get_funding_round(row):
    try:
        # align formatting as "Series A(B, C, E)"
        round_info =  re.search(r'[Ss]eries.[ABCDE]', row["title"]).group(0)
        return "Series {}".format(round_info.split(" ")[1])
        
    except:
        try:
            return re.search(r'[Ss]eries.[ABCDE]', row["excerpt"]).group(0)
            print ("test")
        except:
            return "NA"
        
data["funding_round"] = data.apply(lambda row: get_funding_round(row), axis = 1)

In [9]:
# get money_raied info
def get_money_raised(row):
    try:
        # reformatting as "$100M"
        money = re.search(r'(\D\d{1,}\.*\d*)\s*([Mm])[illion]*', row["title"])
        return money.group(1) + " " + money.group(2).upper()
        
    except:
        try:
            money = re.search(r'(\D\d{1,}\.*\d*)\s*([Mm])[illion]*', row["excerpt"])
            return money.group(1) + " " + money.group(2).upper()
        except:
            return "unknown"

data["money_raised"] = data.apply(lambda row: get_money_raised(row), axis = 1)

In [10]:
# drop money_raised == "NA" AND money_raised == "unknown"
data.drop(data[ (data.funding_round == "NA") & (data.money_raised == "unknown")].index, inplace = True)

In [11]:
sum(data.duplicated())

261

In [12]:
data.drop_duplicates(inplace = True)

In [13]:
data.shape

(646, 6)

In [14]:
data.to_csv("article_after_processing1.csv", index = False)